# Combining and grouping DataFrames

In [ ]:
import numpy as np
import pandas as pd

Two of the most important operations one can do with DataFrames is 1) combine multiple sources of information and 2) efficiently group and summarize information. Both of these topics are vast and here again, we ony present a few important approaches.

## Concatenation

The simplest case where we need to combine multiple sources of information is if those sources are of the same "type", i.e. they have the same columns but different entries. Imagine for example lists of participants to an event broken in muliple files with names starting with A-K, L-S etc. In that case we can just "glue" datasets together or more formally **concatenate** them. Here we load to Excel sheets with composer information:

In [ ]:
file_url = 'https://github.com/guiwitz/NumpyPandas_course/blob/master/Data/composers.xlsx?raw=true'
composers1 = pd.read_excel(file_url, index_col='composer',sheet_name='Sheet1')
composers1

In [ ]:
composers2 = pd.read_excel('https://github.com/guiwitz/NumpyPandas_course/blob/master/Data/composers.xlsx?raw=true', index_col='composer',sheet_name='Sheet3')
composers2

We see that we have the same information in both tables, but for different composers. We can just concetenate them using the ```pd.concat``` function and providing the two DataFrames in a list:

In [ ]:
all_composers = pd.concat([composers1,composers2])

In [ ]:
all_composers

One potential problem is that two tables contain duplicated information, like here the Mahler entry. Pandas offers a lot of useful functions to clean-up data, for example ```drop_duplicates()```:

In [ ]:
all_composers.drop_duplicates()

## Joining two tables

A slightly more complex case is if we have two datasets with (almost) the same items, in our case composers, but with different information that we want to combine. For example:

In [ ]:
composers1 = pd.read_excel(file_url, index_col='composer',sheet_name='Sheet1')
composers1

In [ ]:
composers2 = pd.read_excel(file_url, index_col='composer',sheet_name='Sheet4')
composers2

We cannot concatenate the tables as they have different columns. However we can go through both tables and **merge for each item the information of both tables**. Note that each table has **one** element that doesn't appear in the other table (Shostakovitch and Brahms).

We can do this merging operation with the ```join``` function, which one uses like this ```left_table.join(right_table)```: 



In [ ]:
composers1.join(composers2)

We see that:
1. Pandas used the index to merge all elements
2. Brahms which was missing in ```composers1``` is absent
3. The first name of Shostakovitch is missing since that composer was absent in ```composers2```.

In other terms, ```join``` used all the elements of the ```left``` table and only those matching from the ```right``` table. This is just a default that can be overriden using the ```how``` option.

In total we have four possibilities as illustrated here:

In [ ]:
from IPython.display import Image
Image(url='https://github.com/guiwitz/ISDAwPython_day2/raw/master/images/left_right.jpeg',width=700)

1. keep all elements of the ```rigth``` table and complete with the matching ones from the ```left``` table:

In [ ]:
composers1.join(composers2, how = 'right')

2. Keep all elements of both tables:

In [ ]:
composers1.join(composers2, how = 'outer')

3. Keep **only** elements present in **both tables**:

In [ ]:
composers1.join(composers2, how = 'inner')

## More general approach

Here we used the index to know which items to combined together, but more generally one can use **any** column for merging. The ```pd.merge``` function offers more flexibility in this regard. We whos only one example here:

In [ ]:
composers1 = pd.read_excel(file_url, sheet_name='Sheet1')
composers2 = pd.read_excel(file_url, sheet_name='Sheet6')

In [ ]:
composers1

In [ ]:
composers2

In [ ]:
pd.merge(composers1, composers2, left_on='composer', right_on='last name', how='outer')

The concepts are the same as in the ```join()``` function. The first table (```composers1```) is the ```left``` table, the second one (```composers2```) is the ```right``` table. And then we specify in ```left_on``` and ```right_on``` which columns to use for the merging. Finally, the ```how```option is the same.

## Grouping

Very often when you want to calculate some statistics on a dataset you need to group some of the data. Let's look at our penguin dataset:

In [ ]:
import pandas as pd
penguins = pd.read_csv('https://raw.githubusercontent.com/allisonhorst/palmerpenguins/master/inst/extdata/penguins.csv')

In [ ]:
penguins.head(3)

In [ ]:
penguins.species.unique()

We see that we have data for three different species of Penguins. What if we want to have the average weight of penguins by species ? One solution would be to make a for loop e.g.:

In [ ]:
weights = []
for species in penguins.species.unique():
  mean_w = penguins.loc[penguins['species'] == species,'body_mass_g'].mean()
  weights.append(mean_w)

In [ ]:
weights

Pandas offers a much simpler solution for this kind of operation with the ```.groupby``` function. Here we only specifiy which columns should be used to form groups and it does the work for us:

In [ ]:
grouped_penguins = penguins.groupby('species')

In [ ]:
grouped_penguins

The output is not directly visible. It's a collection of tables grouped according to species. Many functions allow us to get one of these subgroups, e.g.:

In [ ]:
grouped_penguins.get_group('Adelie')

However we wont go here through the details of the group data structure. What we can simply do is magically apply functions directly on the grouped object. For example to get the mean:

In [ ]:
grouped_penguins.mean()

As you can see Pandas automatically computes the mean for each category and each column. The output then is a DataFrame where each line corresponds to a given category. One can push this further by using e.g. multiple columns for grouping, but this goes beyond the present course.

## Exercises

1. Load a first DataFrame that contains information about covid19 vaccination locations: https://github.com/owid/covid-19-data/blob/master/public/data/vaccinations/locations.csv 
2. Load a second DataFrame which contains daily information about vaccination for every country: https://github.com/owid/covid-19-data/blob/master/public/data/vaccinations/vaccinations.csv
3. Create a sub-dataframe of the table in (1) that only contains the fields ```location``` and ```vaccines```.
4. Merge the two tables using the ```location``` field as key for merging. Use once ```left``` merge and once ```right``` marge.
5. Do you see a difference in the result?
5b. Bonus question: knowing that you can use the ```.unique()``` method on columns, and that there's a function called ```np.isin``` that compares two lists, can you figure out which country is missing ?
6. Using the table from (1), group the data by ```location``` and calculate the average number of ```daily_vaccinations```